<a href="https://colab.research.google.com/github/praneethuddagiri/code_snippets/blob/master/BSE_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bsedata
!pip install pandas_datareader
from bsedata.bse import BSE
from pandas_datareader import data as wb

b = BSE(update_codes = True)

In [2]:
!pip install twilio
!pip install --upgrade gspread
import requests
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import smtplib, ssl
from email.mime.multipart import MIMEMultipart
from twilio.rest import Client
from google.colab import drive
from gspread.models import Cell

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

%matplotlib

     |████████████████████████████████| 460kB 2.6MB/s 
  Created wheel for twilio: filename=twilio-6.46.0-py2.py3-none-any.whl size=1213591 sha256=9f0f34c044a6009a4b9823ec48058b0bd45dbb2227a7242cfac228cf9bb93314
  Stored in directory: /root/.cache/pip/wheels/12/8d/ff/55e7cdb44da8bc9b69e4a091a6f0b260bd3ab1e279f9cf271e
Successfully built twilio
  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1
Using matplotlib backend: agg


In [3]:
class Stock_ticks():
    stock_lod = []
    call_count = 0
    def get_stock_vals(self,codelist):
        quotes = []
        for code in codelist:
            #print(code)
            quotes.append(b.getQuote(code))
        return quotes
    
    def get_test_stock_vals(self,df,name):
        test_list = []
        for i in range(len(df)):
            stock_dict_1 = {}
            stock_dict_1['companyName'] = name
            stock_dict_1['currentValue'] = df.loc[i,'Close Price']
            stock_dict_1['52weekLow'] = min(df['Close Price'])
            stock_dict_1['52weekHigh'] = max(df['Close Price'])
            stock_dict_1['updatedOn'] = df.loc[i,'Date']
            stock_dict_1['scripCode'] = df.loc[0,'scripCode']
            if(i<=self.call_count):
                test_list.append(stock_dict_1)
        self.call_count = self.call_count + 1
        #print(test_list)
        return test_list
    
    def add_data(self, quote):
        #for entry in self.stock_lod:
        keys = [entry['Name'] for entry in self.stock_lod]
        #print(keys)
        if(quote['companyName'] in keys):
            for entry in self.stock_lod:
                if(entry['Name'] == quote['companyName']):
                    entry['Values'].append(float(quote['currentValue']))
                    entry['52weekLow'] = float(quote['52weekLow'])
                    entry['52weekHigh'] = float(quote['52weekHigh'])
                    entry['time'].append(quote['updatedOn'])
                    entry['scripCode'] = int(quote['scripCode'])
                    entry['pChange'] = float(quote['pChange'])
        else:
            entry = {}
            entry['Name'] = quote['companyName']
            entry['Values'] = []
            entry['Values'].append(float(quote['currentValue']))
            entry['52weekLow'] = float(quote['52weekLow'])
            entry['52weekHigh'] = float(quote['52weekHigh'])
            entry['time'] = []
            entry['time'].append(quote['updatedOn'])
            entry['scripCode'] = int(quote['scripCode'])
            entry['pChange'] = float(quote['pChange'])
            self.stock_lod.append(entry)
        #print(self.stock_lod)
        
    def add_stock_names(self,codelist):
        for code in codelist:
            stock_dict = {}
            stock_dict['Name'] = ''
            stock_dict['Values'] = []
            stock_dict['52weekLow'] = 0
            stock_dict['52weekHigh'] = 0
            stock_dict['time'] = []
            self.stock_lod.append(stock_dict)
    
    def formatted_print(self,quote):
        print("Company Name: {}".format(quote['companyName']))
        print("currentValue: {}".format(quote['currentValue']))
        print("52weekLow: {}".format(quote['52weekLow']))
        print("52weekHigh: {}".format(quote['52weekHigh']))
        print("updatedOn: {}".format(quote['updatedOn']))
    
    def plot(self):
        fig = figure(num=None, figsize=(20, 20), dpi=80, facecolor='w', edgecolor='k')
        for index,entry in enumerate(self.stock_lod):
            #print(len(self.stock_lod))
            #plt.subplot(len(self.stock_lod)/2,2,index+1)
            plt.title("{}".format(entry['Name']))
            lv = float(entry['52weekLow'])
            hv = float(entry['52weekHigh'])
            y_range = np.arange(lv,hv,0.1)
            plt.xticks(rotation=90)
            sns.lineplot(x=entry['time'],y=entry['Values'],sort=False)
            #plt.pause(5)
            #line.remove()
        plt.tight_layout()
        plt.show()
        plt.pause(10)
        plt.close()
    
    def analyze(self,critera_lod):
        sa = stock_analyze()
        string = sa.analyze(self.stock_lod,critera_lod)
        if(string != ''):
            print(string)
            self.send_mail(string)
            #self.send_sms(string)
        #for index,entry in enumerate(self.stock_lod):
            #if(float(entry['Values'][-1]) < 0.6*(float(entry['52weekHigh']))):
            #    print_String = 'Stock {} went below 70% of 52week High. Present value:{}, 52weekHigh:{}'.format(entry['Name'],float(entry['Values'][-1]),float(entry['52weekHigh']))
            #    print(print_String)
            #    self.send_mail(print_String)
            #elif(float(entry['Values'][-1]) >= (float(entry['52weekHigh']))):
            #    print_String = 'Stock {} Reached 52week High. Present value:{}, 52weekHigh:{}'.format(entry['Name'],float(entry['Values'][-1]),float(entry['52weekHigh']))
            #    print(print_String)
            #    self.send_mail(print_String)
        
    def send_mail(self,data):
        port = 465  # For SSL
        password = '$tellul@r_X@GZY3141'
        sender_email="stock.python.test@gmail.com"
        receiver_email=["praneethuddagiri@gmail.com","kumar.pramod402@gmail.com"]
        # Create a secure SSL context
        context = ssl.create_default_context()
        
        
        msg = MIMEMultipart(data)
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = 'Stock Alert'

        #msg = """From: {}
        #To: {}
        #Subject: Stock Alert\n
        #{}
        #""".format(,,data)
        
        message = 'Subject: {}\n\n{}'.format("Stock Alert", data)

        with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
            server.ehlo
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, message)
            
    def send_sms(self,data):
        account_sid = ""
        auth_token = ""

        client = Client(account_sid, auth_token)

        client.api.account.messages.create(
            to="+919035378897",
            from_="+12624390775",
            body=data)
    
    def __init__(self): 
        self.stock_lod = []
        self.call_count = 0

In [4]:
class stock_analyze():
    #call all other analysis functions here
    def analyze(self,stock_lod,critera_lod):
        string = ''
        string += self.check_criteria(stock_lod,critera_lod)
        #self.went_above()
        return string 
        
    def dict_temp(self):
        my_dict = {}
        my_dict['criteria_name']='default_criteria'
        my_dict['scripCode'] = 0
        my_dict['floor'] = -1.0#low value
        my_dict['ceil'] = 10**6#high value
        my_dict['en_mv_chk'] = 0
        my_dict['ceil_mv'] = +5.0
        my_dict['floor_mv'] = -5.0
        my_dict['cond_hit'] = 0
        return my_dict
        
    def create_went_below_dict(self,scripCode,value,ct_name):
        wb_dict = self.dict_temp()
        wb_dict['scripCode'] = int(scripCode)
        wb_dict['floor'] = float(value)
        wb_dict['criteria_name'] = ct_name
        return wb_dict

    def create_went_above_dict(self,scripCode,value,ct_name):
        wb_dict = self.dict_temp()
        wb_dict['scripCode'] = int(scripCode)
        wb_dict['ceil'] = float(value)
        wb_dict['criteria_name'] = ct_name
        return wb_dict

    def create_mov_dict(self,scripCode,ct_name,ceil_value=5.0,floor_value=5.0):
        wb_dict = self.dict_temp()
        wb_dict['scripCode'] = int(scripCode)
        wb_dict['en_mv_chk'] = 1
        wb_dict['ceil_mv'] = ceil_value
        wb_dict['floor_mv'] = floor_value
        wb_dict['criteria_name'] = ct_name
        return wb_dict

    
    def check_criteria(self,stock_lod,critera_lod):
        string = ''
        all_scrips = [entry['scripCode'] for entry in critera_lod]
        
        for criteria in critera_lod:
            stock_dict = self.get_stock_dict(criteria['scripCode'],stock_lod)
            if((stock_dict['Values'][-1] < criteria['floor']) and (criteria['cond_hit'] != 1)):
                criteria['cond_hit'] = 1
                print('Hit low condition')
                string += 'Criteria:{} Stock value of {} went below floor price of {}. Present value:{}\n'.format(criteria['criteria_name'],stock_dict['Name'],criteria['floor'],stock_dict['Values'][-1])
            if((stock_dict['Values'][-1] > criteria['ceil']) and (criteria['cond_hit'] != 1)):
                criteria['cond_hit'] = 1
                print('Hit high condition')
                string += 'Criteria:{} Stock value of {} went above ceil price of {}. Present value:{}\n'.format(criteria['criteria_name'],stock_dict['Name'],criteria['ceil'],stock_dict['Values'][-1])

            #percentage checks
            if((stock_dict['pChange'] > criteria['ceil_mv']) and (criteria['cond_hit'] != 1)):
                criteria['cond_hit'] = 1
                print('Percentage ceil condition hit')
                string += 'Criteria:{} Stock value of {} saw percentage change above {}. Present value:{}\n'.format(criteria['criteria_name'],stock_dict['Name'],criteria['ceil_mv'],stock_dict['pChange'])
            if((stock_dict['pChange'] < criteria['floor_mv']) and (criteria['cond_hit'] != 1)):
                criteria['cond_hit'] = 1
                print('Percentage ceil condition hit')
                string += 'Criteria:{} Stock value of {} saw percentage change below {}. Present value:{}\n'.format(criteria['criteria_name'],stock_dict['Name'],criteria['floor_mv'],stock_dict['pChange'])
        return string
    
    def get_stock_dict(self,scripCode,stock_lod):
        for entry in stock_lod:
            if(entry['scripCode'] == scripCode):
                return entry
        print('scripCode not found in stock_lod')
        return ''

In [5]:
import pandas as pd
from gspread.models import Cell
class Database():
  col_row = []
  #path = '/content/gdrive/Colab Notebooks/Stock_database.xlsx'
  def read_hist_data(self):
    workbook = gc.open('Stock_database.xlsx')
    worksheet = workbook.worksheet("Hist_Data")
    #worksheet_list = workbook.worksheets()
    #print(worksheet_list)
    # get_all_values gives a list of rows.
    rows = worksheet.get_all_values()
    self.col_row = rows[0]
    if(len(rows) > 1):
      rows= rows[1:]
    hist_df = pd.DataFrame.from_records(rows)
    hist_df.columns = self.col_row
    return hist_df

  @staticmethod
  def search_in_stock_list(stock_lod,stock_code):
    for item in stock_lod:
      if(int(item['scripCode'])==int(stock_code)):
        return item

  def update_present_vals(self, stock_lod):
    cells = []
    workbook = gc.open('Stock_database.xlsx')
    worksheet = workbook.worksheet("NSE_names")
    rows = worksheet.get_all_values()
    col_row = rows[0]
    if(len(rows) > 1):
      rows= rows[1:]
    info_df = pd.DataFrame.from_records(rows)
    info_df.columns = col_row
    codelist = list(info_df['BSE Code'])
    worksheet = workbook.worksheet("Hist_Data")
    #create a lit of present values
    pres_val_list = [self.search_in_stock_list(stock_lod,code)['Values'][-1] for code in codelist]

    cells.append(Cell(row=102, col=1, value='Present Value'))
    for index,item in enumerate(pres_val_list):
      cells.append(Cell(row=102, col=index+2, value=item))

    worksheet.update_cells(cells)

  def write_hist_data(self,df):
    cells = []
    workbook = gc.open('Stock_database.xlsx')
    worksheet = workbook.worksheet("Hist_Data")
    #cells.append(self.col_row)
    #print(list(range(0,len(df)+1)))
    #print(list(range(0,len(self.col_row))))
    for k in range(0,len(self.col_row)):
      cells.append(Cell(row=1, col=k+1, value=self.col_row[k]))
    for i in range(1,len(df)+1):
      for j in range(1,len(self.col_row)+1):
        #print(Cell(row=i+1, col=j+1, value=df.iloc[i,j]))
        cells.append(Cell(row=i+1, col=j, value=(df.iloc[i-1,j-1])))
    #print(cells)
    worksheet.update_cells(cells)

  def update_hist_hdr(self,col_hdr):
    cells = []
    workbook = gc.open('Stock_database.xlsx')
    worksheet = workbook.worksheet("Hist_Data")
    for k in range(0,len(col_hdr)):
      cells.append(Cell(row=1, col=k+2, value=col_hdr[k]))

    #print(cells)

    worksheet.update_cells(cells)

  def read_stock_codes(self):
    workbook = gc.open('Stock_database.xlsx')
    worksheet = workbook.worksheet("BSE_codes")
    rows = worksheet.get_all_values()
    self.col_row = rows[0]
    code_df = pd.DataFrame.from_records(rows)
    code_df.columns = self.col_row
    return code_df

In [6]:
class Database_new():

  def __init__(self):
    pass
    #self.bse_stock_list = [i+'.BS' for i in stock_list]
    #self.nse_stock_list = [i+'.NS' for i in stock_list]


  @staticmethod
  def write_to_workbook(df, sheetname):
    cells = []
    workbook = gc.open('Stock_database.xlsx')
    worksheet = workbook.worksheet(sheetname)
    col_row = list(df.columns)
    for k in range(0,len(col_row)):
      cells.append(Cell(row=1, col=k+1, value=col_row[k]))
    for i in range(1,len(df)+1):
      for j in range(1,len(col_row)+1):
        #print(Cell(row=i+1, col=j+1, value=df.iloc[i,j]))
        if(j==1):
          date_string = df.iloc[i-1,j-1].strftime("%m/%d/%Y")
          cells.append(Cell(row=i+1, col=j, value=date_string))
          continue
        cells.append(Cell(row=i+1, col=j, value=(df.iloc[i-1,j-1])))
    #print(cells)
    worksheet.update_cells(cells)

  @staticmethod
  def create_returns_df(df):
    ret_df = df.iloc[:,:101]
    ret_df['Date'] = list(range(1,101))
    #ret_df = ret_df/ret_df.shift(1)
    ret_df = 100*ret_df/ret_df.iloc[0,:]
    ret_df['Date'] = df['Date'][:101]
    #ret_df = ret_df.drop(0)
    #print(ret_df)
    return ret_df

  def write_hist_data(self, df):
    cells = []
    #workbook = gc.open('Stock_database.xlsx')
    #worksheet = workbook.worksheet("Hist_Data")
    print(df)
    self.write_to_workbook(df,'Hist_Data')

    #start creating other DataFrame here from Base DataFrame
    ret_df = self.create_returns_df(df)
    self.write_to_workbook(ret_df,'Returns_Data')

    #cells.append(self.col_row)
    #print(list(range(0,len(df)+1)))
    #print(list(range(0,len(self.col_row))))
    

  def update_hist_data(self):
    samp_count = 100
    self.read_stock_names()
    stock_list = ['Date']
    stock_list.extend(self.nse_stock_list)
    print(stock_list)
    df = pd.DataFrame(columns=stock_list)
    stock_df = wb.DataReader(self.nse_stock_list[0],data_source='yahoo',start='1-1-2000')
    df['Date'] = list(stock_df.index)[-1*samp_count:]
    for stock in self.nse_stock_list:
      stock_df = wb.DataReader(stock,data_source='yahoo',start='1-1-2000')
      close_price_list = list(stock_df['Adj Close'][-1*samp_count:])#get only last hundred entries
      df.loc[:,stock] = close_price_list

    self.write_hist_data(df)

  def read_stock_names(self):
    workbook = gc.open('Stock_database.xlsx')
    worksheet = workbook.worksheet("NSE_names")
    rows = worksheet.get_all_values()
    self.col_row = rows[0]
    names_df = pd.DataFrame.from_records(rows)
    names_df.columns = list(self.col_row)
    self.nse_stock_list = list(names_df.iloc[1:,1])
    self.nse_stock_list = [str(i)+'.NS' for i in self.nse_stock_list]
    print(self.nse_stock_list)

In [ ]:
#codelist = ['500325','532215','500209','500112','532281','532174']#[RIL,Axis,Infy,SBI,HCL,ICICI]
sc = Stock_ticks()
db = Database()
code_df = db.read_stock_codes()
codelist = list(code_df['BSE code'])[1:]
db_new = Database_new()
db_new.update_hist_data()
#db.update_hist_hdr(codelist)
#hist_df = db.read_hist_data()
hist_index = 6 #29-sep-2020
quotes = []

#drive.mount('/content/gdrive')
#create check criteria
criteria_lod = []
sa = stock_analyze()
criteria_lod = get_all_criteria(sa,criteria_lod)
#while 1:
for i in range(72):
    quotes = sc.get_stock_vals(codelist)
    #tmp_quotes = sc.get_test_stock_vals(ril_df,'RIL')
    #quotes.extend(tmp_quotes)
    #tmp_quotes = sc.get_test_stock_vals(axis_df,'AXIS')
    #quotes.extend(tmp_quotes)
    #print(len(quotes))
    for quote in quotes:
        #print(quote)
        sc.add_data(quote)
        #hist_df.loc[hist_index,quote['scripCode']] = quote['currentValue']
    #print(hist_df)
    db.update_present_vals(sc.stock_lod)
    #print('Updated at:{}'.format(quotes[0]['updatedOn']))
    #db.write_hist_data(hist_df)
    #if(i > 10):
        #sc.plot()
    #sc.formatted_print()
    sc.analyze(criteria_lod)
    time.sleep(100)

['RELIANCE.NS', 'AXISBANK.NS', 'SBIN.NS', 'ICICIBANK.NS', 'HCLTECH.NS', 'INFY.NS', 'BHARTIARTL.NS', 'SBICARD.NS', 'JSWSTEEL.NS', 'TCS.NS', 'DABUR.NS', 'HDFC.NS', 'HEROMOTOCO.NS']
['Date', 'RELIANCE.NS', 'AXISBANK.NS', 'SBIN.NS', 'ICICIBANK.NS', 'HCLTECH.NS', 'INFY.NS', 'BHARTIARTL.NS', 'SBICARD.NS', 'JSWSTEEL.NS', 'TCS.NS', 'DABUR.NS', 'HDFC.NS', 'HEROMOTOCO.NS']
         Date  RELIANCE.NS  AXISBANK.NS  ...    DABUR.NS      HDFC.NS  HEROMOTOCO.NS
0  2020-06-01  1514.662720   396.950012  ...  467.759155  1699.768677    2305.380127
1  2020-06-02  1529.955200   410.100006  ...  459.385834  1773.963135    2297.402344
2  2020-06-03  1535.883057   409.549988  ...  461.877899  1795.580688    2279.514648
3  2020-06-04  1573.890381   394.350006  ...  461.080414  1725.788208    2308.303467
4  2020-06-05  1575.783203   405.299988  ...  461.379486  1728.037842    2319.600830
..        ...          ...          ...  ...         ...          ...            ...
95 2020-10-13  2280.699951   461.350006

In [7]:
def get_all_criteria(sa,criteria_lod):
  criteria_dict = sa.create_went_above_dict(500325,2300,'ril_above_2300')
  criteria_lod.append(criteria_dict)
  criteria_dict = sa.create_went_above_dict(532281,880,'hcl_above_880')
  criteria_lod.append(criteria_dict)
  criteria_dict = sa.create_went_below_dict(543066,800,'sbi_cards_below_800')
  criteria_lod.append(criteria_dict)
  criteria_dict = sa.create_went_below_dict(532215,420,'axis_below_420',)
  criteria_lod.append(criteria_dict)
  return criteria_lod